## 1. Semantic Kernel Introduction

### Overview of Semantic Kernel (SK) and Its Importance

**Semantic Kernel** is an open-source SDK from Microsoft that acts as middleware between your application code and AI large language models (LLMs). It enables developers to easily integrate AI into apps by letting AI agents call code functions and by orchestrating complex tasks. SK is *lightweight* and *modular*, designed for **enterprise-grade solutions** with features like telemetry and filters for responsible AI. Major companies (including Microsoft) leverage SK because it’s flexible and **future-proof** – you can swap in new AI models as they emerge without rewriting your code. In short, SK helps build **robust, scalable AI applications** that can evolve with advancing AI capabilities.

Key reasons why Semantic Kernel is important for AI application development:

- **Bridging AI and Code**: SK combines natural language **prompts** with your **existing code and APIs**, allowing AI to take actions. The AI can request a function call and SK will execute that function and return results back to the model. This bridges the gap between what the AI *intends* and what your code can do.
- **Plugins (Skills)**: You can expose functionalities (from simple math to complex business logic or external APIs) as SK **plugins**. By describing your code to the AI (via function definitions), the model can invoke these functions to fulfill user requests. This plugin architecture makes your AI solutions **modular and extensible**.
- **Enterprise-ready**: SK includes support for **security, observability, and compliance** (e.g. integration with Azure services, monitoring, content filtering). Hooks and filters ensure you can enforce policies (for instance, prevent sensitive data leakage).
- **Multi-modal & Future-Proof**: SK natively supports multiple AI services (OpenAI, Azure OpenAI, HuggingFace, etc.) and modalities. Chat-based APIs can be extended to voice or other modes. As new models (like vision-enabled models or better language models) come out, SK lets you plug them in without major changes.
- **Rapid Development**: By handling the heavy lifting of prompt orchestration, function calling, and memory management, SK enables faster development of AI features. You focus on defining *what* you want the AI to do (skills, prompts) and SK handles *how* to do it. Microsoft claims that SK helps “deliver AI solutions faster than any other SDK” due to its ability to **automatically call functions**.

---

### Services and Core Components of SK

Semantic Kernel's architecture revolves around a few core components and services:

- **Kernel**: The central object that orchestrates everything. The `Kernel` holds configuration for AI services, manages plugins (skills), coordinates function calls, and maintains contextual state (memory). You typically create one Kernel instance in your app and use it to register functions and perform AI queries.
- **AI Services**: SK connects to AI models for different tasks:
  - *Chat Models*: e.g. Azure OpenAI GPT-4o-mini or GPT-4o for natural language generation and understanding.
  - *Embedding Models*: for converting text to vector embeddings (used in memory/search).
  - *Other Modalities*: connectors for images, speech, etc., if needed.
  
  You configure the Kernel with the endpoints/keys for the services you need. For example, adding an Azure OpenAI chat completion service:

---

In [ ]:
!pip install semantic-kernel python-dotenv --quiet

In [2]:
import semantic_kernel as sk
import os
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from dotenv import load_dotenv

load_dotenv('../.env', override=True)

kernel = sk.Kernel()


deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")


chat_completion = AzureChatCompletion(
        endpoint=base_url,    
        deployment_name=deployment_name,
        api_key=api_key,
    )

kernel.add_service(chat_completion)

You have now successfully created a kernel and added a chat completion service.

Similarly, you can add an embedding generation service via `kernel.add_service(text_embedding)` if performing semantic memory searches. But don't worry we will dive into this at a later stage.

---

### Functions and Plugins in SK

**Functions** in Semantic Kernel are the actions that the AI can perform. They come in two types:

- **Semantic Functions**: Backed by a prompt and LLM. For example, a function `TranslateToFrench` might use the prompt `"Translate this to French: {{$input}}"`.
- **Native Functions**: Backed by code. For example, a function `SendEmail(to, subject, body)` that uses an API to send an email.

These functions are typically grouped into **Plugins** (or "Skills"). Grouping functions into plugins helps manage and control which capabilities are exposed to the AI.

**Using Plugins/Functions**: Once registered with the kernel (via `kernel.add_function` or `kernel.add_plugin`), functions become available for invocation. They can be called directly in code via `kernel.invoke(function, input)`, or the AI model can automatically choose to invoke them as needed.

SK’s plugin system is highly flexible:
- You can load **OpenAPI** specifications or API endpoints as plugins.
- Plugins can be shared across projects, allowing organizations to build a library of useful AI plugins.

---

You can add plugins to the Kernel in various ways:

- **Inline Definition**: Define a prompt or function in code and register it.
- **From Files or Classes**: Load plugins from directories or Python classes decorated appropriately.

For example, to add a simple **semantic function** inline:

In [3]:
# Define a semantic function (prompt) to generate a TL;DR summary
prompt_template = "{{$input}}\n\nTL;DR in one sentence:"

summarize_fn = kernel.add_function(
    prompt=prompt_template, 
    function_name="tldr", 
    plugin_name="Summarizer",
    max_tokens=50)

# Use the function
long_text = """
Semantic Kernel is a lightweight, open-source development kit that lets 
you easily build AI agents and integrate the latest AI models into your C#, 
Python, or Java codebase. It serves as an efficient middleware that enables 
rapid delivery of enterprise-grade solutions.
"""

summary = await kernel.invoke(summarize_fn, input=long_text)
print(summary)

Semantic Kernel is an open-source toolkit enabling easy integration of AI models into C#, Python, or Java codebases for building AI agents and enterprise-grade solutions.


## Exercise 1

Write a semantic function, taken a text and a target language it can provide the translation of this text in the target laanguage

In [ ]:
# Define a semantic function (prompt) to generate a TL;DR summary
prompt_template = "{{$input}}\n\nTranslate this into {{$target_lang}}:"

translate_fn = kernel.add_function(
    prompt=prompt_template, 
    function_name="translator", 
    plugin_name="Translator",
    max_tokens=50)

# Use the function
text = """
Semantic Kernel is a lightweight, open-source development kit that lets 
you easily build AI agents and integrate the latest AI models into your C#, 
Python, or Java codebase. It serves as an efficient middleware that enables 
rapid delivery of enterprise-grade solutions.
"""

summary = await kernel.invoke(translate_fn, input=text, target_lang="French")
print(summary)

Semantic Kernel est un kit de développement léger et open-source qui vous permet de créer facilement des agents d'IA et d'intégrer les derniers modèles d'IA dans votre code en C#, Python ou Java. Il sert de middleware efficace permettant une livraison



Now if we want to build a Plugin with a set of **native functions** we can do the following:

In [3]:
from typing import TypedDict, Annotated, List, Optional
from semantic_kernel.functions.kernel_function_decorator import kernel_function

class LightModel(TypedDict):
   id: int
   name: str
   is_on: bool | None
   brightness: int | None
   hex: str | None

class LightsPlugin:
   def __init__(self, lights: list[LightModel]):
      self.lights = lights
   

   @kernel_function
   async def get_lights(self) -> List[LightModel]:
      """Gets a list of lights and their current state."""
      return self.lights

   @kernel_function
   async def get_state(
      self,
      id: Annotated[int, "The ID of the light"]
   ) -> Optional[LightModel]:
      """Gets the state of a particular light."""
      for light in self.lights:
         if light["id"] == id:
               return light
      return None

   @kernel_function
   async def change_state(
      self,
      id: Annotated[int, "The ID of the light"],
      new_state: LightModel
   ) -> Optional[LightModel]:
      """Changes the state of the light."""
      for light in self.lights:
         if light["id"] == id:
               light["is_on"] = new_state.get("is_on", light["is_on"])
               light["brightness"] = new_state.get("brightness", light["brightness"])
               light["hex"] = new_state.get("hex", light["hex"])
               return light
      return None

In [4]:
# Create dependencies for the plugin
lights = [
    {"id": 1, "name": "Table Lamp", "is_on": False, "brightness": 100, "hex": "FF0000"},
    {"id": 2, "name": "Porch light", "is_on": False, "brightness": 50, "hex": "00FF00"},
    {"id": 3, "name": "Chandelier", "is_on": True, "brightness": 75, "hex": "0000FF"},
]

plugin = LightsPlugin(lights=lights)


kernel.add_plugin(
   plugin=plugin,
   plugin_name="Lights",
)

KernelPlugin(name='Lights', description=None, functions={'change_state': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='change_state', plugin_name='Lights', description='Changes the state of the light.', parameters=[KernelParameterMetadata(name='id', description='The ID of the light', default_value=None, type_='int', is_required=True, type_object=<class 'int'>, schema_data={'type': 'integer', 'description': 'The ID of the light'}, include_in_function_choices=True), KernelParameterMetadata(name='new_state', description=None, default_value=None, type_='LightModel', is_required=True, type_object=<class '__main__.LightModel'>, schema_data={'type': 'object', 'properties': {'id': {'type': 'integer'}, 'name': {'type': 'string'}, 'is_on': {'type': ['boolean', 'null']}, 'brightness': {'type': ['integer', 'null']}, 'hex': {'type': ['string', 'null']}}, 'required': ['id', 'name']}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelPa

In [5]:
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()
history.add_user_message("Please turn on the lamp")

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

# Print the results
print("Assistant > " + str(result))

# Add the message from the agent to the chat history
history.add_message(result)


Assistant > The Table Lamp is already turned on. If you need any adjustments or further actions, feel free to let me know!


### Automatic Function Calling

One of the most powerful features of Semantic Kernel is its ability to **automatically orchestrate multi-step operations** by calling multiple functions in sequence. In this example, a **LightsPlugin** is defined with three asynchronous native functions:

- **get_lights()**: Retrieves the list of lights along with their current states.
- **get_state(id)**: Returns the state of a specific light, given its ID.
- **change_state(id, new_state)**: Changes the state of a specified light to a new state (for example, turning it on, adjusting brightness, or changing its color).

**How It Works**:
- The kernel exposes all registered functions to the AI model.
- When a user issues a command like *"Turn on all the lights and give me their final state,"* the AI model analyzes the request and plans the necessary steps:
  1. Call **get_lights()** to retrieve all available lights.
  2. For each light, invoke **change_state()** with a new state (e.g., setting `"is_on": True`).
  3. Optionally, call **get_state()** for each light to confirm the updated status.
- The kernel then returns a comprehensive result that reflects the final state of each light.

**Example Scenario**:
- **User Query**: *"Turn on all the lights and tell me their status."*
- **Step 1**: The system calls `LightsPlugin.get_lights()` to fetch the current list of lights.
- **Step 2**: It iterates over the list and calls `LightsPlugin.change_state(id, new_state)` to turn each light on.
- **Step 3**: Finally, it may call `LightsPlugin.get_state(id)` for each light to confirm the changes.
- The final output, including the updated state for each light, is returned to the user.

This automatic orchestration simplifies the management of multi-step tasks, enabling the AI to autonomously plan and execute function calls without manual intervention.

In [6]:
## TODO: Exercise

# Add the message from the user to the chat history
# history.add_user_message("Please turn on all the lamps")


# history.add_user_message("Please turn off all the lamps and give me their final state")

### Filters in Semantic Kernel

Filters provide a layer of control and visibility over function execution, ensuring responsible AI practices and enterprise-grade security. They allow you to:

- **Validate Permissions:**  
  For example, a filter can check user permissions before initiating an approval flow.

- **Intercept Function Execution:**  
  - **Function Invocation Filter:**  
    Runs every time a function is called; it can access function details, handle exceptions, override results (e.g., for caching or responsible AI), or retry on failure.
  - **Prompt Render Filter:**  
    Triggered before a prompt is rendered; it allows you to view or modify the prompt and even override the result to prevent submission.
  - **Auto Function Invocation Filter:**  
    Works within automatic function calling, providing additional context (like chat history and iteration counters) and can terminate the process early if needed.

Each filter receives a context object with execution details and must call the next delegate (or callback) to continue the execution chain. Filters can be registered either by using the `add_filter` method on the Kernel or via the `@kernel.filter` decorator.


One of the things that I would like to improve in our plugin implementation is to add debugging, that way I can integrate with external systems for auditing purposes

Lets implement that

In [10]:
import logging

# Configure the logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [11]:
from typing import Awaitable, Callable
from semantic_kernel.filters import FunctionInvocationContext

logger = logging.getLogger(__name__)


async def logger_filter(context: FunctionInvocationContext, next: Callable[[FunctionInvocationContext], Awaitable[None]]) -> None:
    logger.info(f"FunctionInvoking - {context.function.plugin_name}.{context.function.name}")

    await next(context)

    logger.info(f"FunctionInvoked - {context.function.plugin_name}.{context.function.name}")

kernel.add_filter('function_invocation', logger_filter)


In [13]:
history.add_user_message("Please turn on all the lamps and give me their final state")

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

# Add the message from the agent to the chat history
history.add_message(result)


# Print the results
print("Assistant > " + str(result))

2025-02-27 14:33:14,725 - INFO - HTTP Request: POST https://aoai-sweden-gbb-dev.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-10-21 "HTTP/1.1 200 OK"
2025-02-27 14:33:14,729 - INFO - OpenAI usage: CompletionUsage(completion_tokens=117, prompt_tokens=903, total_tokens=1020, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-02-27 14:33:14,731 - INFO - processing 3 tool calls in parallel.
2025-02-27 14:33:14,732 - INFO - Calling Lights-change_state function with args: {"id": 1, "new_state": {"id": 1, "name": "Table Lamp", "is_on": true}}
2025-02-27 14:33:14,734 - INFO - Function Lights-change_state invoking.
2025-02-27 14:33:14,735 - INFO - FunctionInvoking - Lights.change_state
2025-02-27 14:33:14,736 - INFO - FunctionInvoking - Lights.change_state
2025-02-27 14:33:14,736 - INFO - F

Assistant > All the lamps have been turned on. Here is their current state:

1. **Table Lamp**: 
   - State: On 
   - Brightness: 100 
   - Color: #FF0000

2. **Porch Light**: 
   - State: On 
   - Brightness: 50 
   - Color: #00FF00

3. **Chandelier**: 
   - State: On 
   - Brightness: 75 
   - Color: #0000FF

If you need any further assistance, just let me know!


### Filters in SK and Their Use Cases

To summarize, in any AI application, it’s important to control input/output and function execution for security, privacy, and correctness. **Filters** in Semantic Kernel act as middleware or interceptors in the execution pipeline.

**Use Cases for Filters**:
- **Security/Policy**: Prevent sensitive data from being sent to the AI.
- **Validation**: Check function arguments before execution.
- **Error Handling**: Catch exceptions and provide default results.
- **Logging/Monitoring**: Log each function call and its response.
- **Post-processing**: Modify outputs before they’re returned to the AI.

- **Context**: The Kernel maintains a context that holds variables (including memory) and the history of the conversation. This context is essential for chaining function calls and maintaining continuity.
- **Planner (Plans)**: SK includes a *planner* that can take a high-level goal and figure out a sequence of function calls to achieve it. This is useful for dynamic task orchestration.
- **Memory**: SK provides a **semantic memory** store for long-term information. Memory is typically backed by a **vector database** or search index. This enables Retrieval-Augmented Generation (RAG) scenarios by allowing the AI to recall relevant information from stored data.
